In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import drive          
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        imgResized = cv2.resize(img, (128,128))
        if img is not None:
            images.append(imgResized)
    return images
folder="/content/drive/My Drive/Hack_D_covid_Dataset/train"

In [4]:

xtrain = load_images_from_folder(folder)

In [5]:
import numpy as np
X = np.array(xtrain)

In [6]:
X.shape

(6923, 128, 128, 3)

In [7]:
import pandas as pd

In [8]:
ytrain = pd.read_csv('/content/drive/My Drive/Hack_D_covid_Dataset/train.csv')

In [9]:
ytrain.head()

,ImageID,Target
0,I7708320312,False
1,I4617585905,True
2,I5897321666,False
3,I3081830511,True
4,I6997670848,False


In [10]:
Ytrain = ytrain.values.tolist()

In [11]:
Y = []
for i in Ytrain:
  if i[1] == False:
    Y.append(0)
  else:
    Y.append(1)

In [12]:
Y = np.array(Y).reshape(6923,1)

In [13]:
Y.shape

(6923, 1)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)

In [15]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(5884, 128, 128, 3)
(5884, 1)
(1039, 128, 128, 3)
(1039, 1)


In [16]:
X_train = X_train/255
X_val = X_val/255
#normalize image vector

In [17]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline


In [23]:
def CovidModel(input_shape):

  X_input = Input(input_shape)
  X = ZeroPadding2D((3,3))(X_input)
  X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
  X = BatchNormalization(axis = 3, name = 'bn0')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool0')(X)
  
  X = ZeroPadding2D((3,3))(X)
  X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv1')(X)
  X = BatchNormalization(axis = 3, name = 'bn1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool1')(X)
  
  X = ZeroPadding2D((3,3))(X)
  X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv2')(X)
  X = BatchNormalization(axis = 3, name = 'bn2')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool2')(X)

  X = ZeroPadding2D((3,3))(X)
  X = Conv2D(128, (7, 7), strides = (1, 1), name = 'conv3')(X)
  X = BatchNormalization(axis = 3, name = 'bn3')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2, 2), strides = (2, 2), name = 'Max_Pool3')(X)
  
  X = Flatten()(X)
  X = Dense(1, activation = 'sigmoid', name = 'fc')(X)
  model = Model(inputs = X_input, outputs = X, name = 'CovidModel')

  return model

In [24]:
covidmodel = CovidModel(X_train.shape[1:])

In [25]:
covidmodel.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ["accuracy"])

In [26]:
covidmodel.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 16)

Epoch 1/20
368/368 [==============================] - 9s 24ms/step - loss: 0.8641 - accuracy: 0.5376
Epoch 2/20
368/368 [==============================] - 9s 24ms/step - loss: 0.7305 - accuracy: 0.5491
Epoch 3/20
368/368 [==============================] - 9s 24ms/step - loss: 0.7113 - accuracy: 0.5561
Epoch 4/20
368/368 [==============================] - 9s 24ms/step - loss: 0.6957 - accuracy: 0.5682
Epoch 5/20
368/368 [==============================] - 9s 24ms/step - loss: 0.6883 - accuracy: 0.5746
Epoch 6/20
368/368 [==============================] - 9s 23ms/step - loss: 0.6778 - accuracy: 0.5906
Epoch 7/20
368/368 [==============================] - 9s 23ms/step - loss: 0.6762 - accuracy: 0.5948
Epoch 8/20
368/368 [==============================] - 9s 23ms/step - loss: 0.6745 - accuracy: 0.5979
Epoch 9/20
368/368 [==============================] - 9s 24ms/step - loss: 0.6739 - accuracy: 0.5964
Epoch 10/20
368/368 [==============================] - 9s 24ms/step - loss: 0.6734 - accura

In [27]:
preds = covidmodel.evaluate(x = X_val, y = Y_val)
print()
print ("Loss = " + str(preds[0]))
print ("Validation Accuracy = " + str(preds[1]))

33/33 [==============================] - 1s 16ms/step - loss: 0.6787 - accuracy: 0.5987

Loss = 0.6787161231040955
Validation Accuracy = 0.5986525416374207


In [28]:
folder="/content/drive/My Drive/Hack_D_covid_Dataset/test"
xtest = load_images_from_folder(folder)

In [29]:
df = pd.read_csv('/content/drive/My Drive/Hack_D_covid_Dataset/test.csv')

In [30]:
X_test = np.array(xtest)

In [31]:
X_test.shape

(1730, 128, 128, 3)

In [35]:
Y_test = covidmodel.predict(X_test)

In [36]:
Y_test.shape

(1730, 1)

In [37]:
Target = []
for i in Y_test:
  if i == 0:
    Target.append(False)
  else:
    Target.append(True)

In [39]:
Target

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,


In [40]:
df['Target'] = Target

In [41]:
df

,ImageID,Target
0,I3202892238,True
1,I9494991764,True
2,I6697447943,True
3,I9118442021,True
4,I9591897319,True
...,...,...
1725,I8250253286,True
1726,I2029298407,False
1727,I5311602610,True
1728,I3138069664,True


In [42]:
df.to_csv('/content/drive/My Drive/Hack_D_covid_Dataset/Fianl_Output.csv')